In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

shl_intern_hiring_assessment_path = kagglehub.competition_download('shl-intern-hiring-assessment')
vishalkochar_shl_transcripts_path = kagglehub.dataset_download('vishalkochar/shl-transcripts')

print('Data source import complete.')


In [ ]:
pip install -U openai-whisper

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [ ]:
import whisper
import torch
import os
import pandas as pd
import warnings
from tqdm import tqdm
from multiprocessing import Pool

warnings.filterwarnings("ignore", category=FutureWarning)

In [ ]:
# === CONFIG ===
NUM_BATCHES = 20
available_devices = ["cuda:0", "cuda:1"]
audio_folder = "/kaggle/input/shl-intern-hiring-assessment/dataset/audios_train"
output_dir = "."  # or any desired path for saving CSVs

# List audio files
audio_files = sorted([f for f in os.listdir(audio_folder) if f.endswith(".wav")])
total_files = len(audio_files)

# Split into 20 batches
def chunkify(lst, n):
    k, m = divmod(len(lst), n)
    return [lst[i*k + min(i, m):(i+1)*k + min(i+1, m)] for i in range(n)]

batches = chunkify(audio_files, NUM_BATCHES)

# Transcription function
def transcribe_file(args):
    file_name, device = args
    audio_path = os.path.join(audio_folder, file_name)

    model = whisper.load_model("medium").to(device)

    result = model.transcribe(
        audio_path,
        fp16=True,
        task="transcribe",
        language="en",
        condition_on_previous_text=False,
        initial_prompt=None,
        temperature=0.0,
        no_speech_threshold=0.0,
        logprob_threshold=-1.0,
        compression_ratio_threshold=10.0
    )

    return {
        "filename": file_name,
        "transcription": result["text"]
    }

# Process all batches
all_transcriptions = []

for batch_idx, batch_files in enumerate(batches, 1):
    print(f"\n Processing batch {batch_idx}/{NUM_BATCHES} with {len(batch_files)} files...")

    args_list = [
        (file, available_devices[i % len(available_devices)])
        for i, file in enumerate(batch_files)
    ]

    with Pool(processes=len(available_devices)) as pool:
        batch_transcriptions = list(tqdm(pool.imap(transcribe_file, args_list), total=len(args_list)))

    # Save batch results
    df_batch = pd.DataFrame(batch_transcriptions)
    batch_csv_path = os.path.join(output_dir, f"transcriptions_batch_{batch_idx}.csv")
    df_batch.to_csv(batch_csv_path, index=False)
    print(f"Saved batch {batch_idx} to {batch_csv_path}")

    # Append to all results and save cumulative CSV
    all_transcriptions.extend(batch_transcriptions)
    df_all = pd.DataFrame(all_transcriptions)
    all_csv_path = os.path.join(output_dir, f"transcriptions_up_to_batch_{batch_idx}.csv")
    df_all.to_csv(all_csv_path, index=False)
    print(f"Saved cumulative transcription → {all_csv_path}")

print("\n All batches completed successfully!")


In [ ]:
# === CONFIG ===
NUM_BATCHES = 10
available_devices = ["cuda:0", "cuda:1"]
audio_folder = "/kaggle/input/shl-intern-hiring-assessment/dataset/audios_test"  # <-- Changed folder
output_dir = "."  # where to save the results
# ==============

# List audio files
audio_files = sorted([f for f in os.listdir(audio_folder) if f.endswith(".wav")])
total_files = len(audio_files)

# Split into 10 batches
def chunkify(lst, n):
    k, m = divmod(len(lst), n)
    return [lst[i*k + min(i, m):(i+1)*k + min(i+1, m)] for i in range(n)]

batches = chunkify(audio_files, NUM_BATCHES)

# Transcription function
def transcribe_file(args):
    file_name, device = args
    audio_path = os.path.join(audio_folder, file_name)

    model = whisper.load_model("medium").to(device)

    result = model.transcribe(
        audio_path,
        fp16=True,
        task="transcribe",
        language="en",
        condition_on_previous_text=False,
        initial_prompt=None,
        temperature=0.0,
        no_speech_threshold=0.0,
        logprob_threshold=-1.0,
        compression_ratio_threshold=10.0
    )

    return {
        "filename": file_name,
        "transcription": result["text"]
    }

# Process all batches
all_transcriptions = []

for batch_idx, batch_files in enumerate(batches, 1):
    print(f"\n Processing TEST batch {batch_idx}/{NUM_BATCHES} with {len(batch_files)} files...")

    args_list = [
        (file, available_devices[i % len(available_devices)])
        for i, file in enumerate(batch_files)
    ]

    with Pool(processes=len(available_devices)) as pool:
        batch_transcriptions = list(tqdm(pool.imap(transcribe_file, args_list), total=len(args_list)))

    # Save batch results
    df_batch = pd.DataFrame(batch_transcriptions)
    batch_csv_path = os.path.join(output_dir, f"transcriptions_test_batch_{batch_idx}.csv")
    df_batch.to_csv(batch_csv_path, index=False)
    print(f"Saved batch {batch_idx} to {batch_csv_path}")

    # Append to all results and save cumulative CSV
    all_transcriptions.extend(batch_transcriptions)
    df_all = pd.DataFrame(all_transcriptions)
    all_csv_path = os.path.join(output_dir, f"transcriptions_test_up_to_batch_{batch_idx}.csv")
    df_all.to_csv(all_csv_path, index=False)
    print(f"Saved cumulative transcription → {all_csv_path}")

print("\n All TEST batches completed successfully!")


In [ ]:
df_train = pd.read_csv("/kaggle/input/shl-intern-hiring-assessment/dataset/train.csv")
df_train.head()

,filename,label
0,audio_1261.wav,1.0
1,audio_942.wav,1.5
2,audio_1110.wav,1.5
3,audio_1024.wav,1.5
4,audio_538.wav,2.0


In [ ]:
df_test = pd.read_csv("/kaggle/input/shl-intern-hiring-assessment/dataset/test.csv")
df_test.head()

,filename
0,audio_706.wav
1,audio_800.wav
2,audio_68.wav
3,audio_1267.wav
4,audio_683.wav


In [ ]:
# — CONFIG —
# Paths to your original target CSVs
TRAIN_META = "/kaggle/input/shl-intern-hiring-assessment/dataset/train.csv"
TEST_META  = "/kaggle/input/shl-intern-hiring-assessment/dataset/test.csv"

# Paths to your cumulative transcription outputs
TRAIN_TRANS_CUM = "/kaggle/input/shl-transcripts/transcriptions_up_to_batch_20.csv"        # from your train script
TEST_TRANS_CUM  = "/kaggle/input/shl-transcripts/transcriptions_test_up_to_batch_10.csv"   # from your test script

# Output filenames
TRAIN_READY = "train_ready.csv"
TEST_READY  = "test_ready.csv"
# — end CONFIG —

# 1) Load metadata
df_meta_train = pd.read_csv(TRAIN_META)   # columns: [filename, label]
df_meta_test  = pd.read_csv(TEST_META)    # columns: [filename]

# 2) Load transcriptions
df_trans_train = pd.read_csv(TRAIN_TRANS_CUM)  # [filename, transcription]
df_trans_test  = pd.read_csv(TEST_TRANS_CUM)   # [filename, transcription]

# 3) Merge train
df_train_ready = (
    df_meta_train
    .merge(df_trans_train, on="filename", how="left")
    .loc[:, ["filename", "transcription", "label"]]
)
df_train_ready.to_csv(TRAIN_READY, index=False)
print(f"Saved train‑ready data → {TRAIN_READY}")

# 4) Merge test
df_test_ready = (
    df_meta_test
    .merge(df_trans_test, on="filename", how="left")
    .loc[:, ["filename", "transcription"]]
)
df_test_ready.to_csv(TEST_READY, index=False)
print(f"Saved test‑ready data → {TEST_READY}")

✅ Saved train‑ready data → train_ready.csv
✅ Saved test‑ready data → test_ready.csv


In [ ]:
df_train_ready.head()

,filename,transcription,label
0,audio_1261.wav,My favorite hobby is cultivation of plants su...,1.0
1,audio_942.wav,The playground looks like very clear and neat...,1.5
2,audio_1110.wav,My goal is to become an electrical employee. ...,1.5
3,audio_1024.wav,My favorite place is in Andhra Pradesh. It is...,1.5
4,audio_538.wav,my favorite places my favorite places my expe...,2.0


In [ ]:
import os
import torch
import pandas as pd
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, accuracy_score
import numpy as np

In [ ]:
# Configuration
MODEL_NAME = 'roberta-large'
NUM_LABELS = 1  # Regression task
MAX_LENGTH = 512
CHUNK_SIZE = 150  # Each chunk will have approximately 150 characters
OVERLAP = 50     # Overlap between chunks
BATCH_SIZE = 8
EPOCHS = 5
LEARNING_RATE = 2e-6
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize tokenizer and model
tokenizer = RobertaTokenizer.from_pretrained(MODEL_NAME)
model = RobertaForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=NUM_LABELS).to(DEVICE)

# Custom dataset class
class GrammarDataset(Dataset):
    def __init__(self, texts, scores, tokenizer, max_length, chunk_size, overlap):
        self.texts = [str(t) if not pd.isna(t) else "" for t in texts]
        self.scores = scores
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.chunk_size = chunk_size
        self.overlap = overlap
        self.data = self._prepare_data()

    def _prepare_data(self):
        data = []
        for text, score in zip(self.texts, self.scores):
            chunks = self.split_into_chunks(text)
            for chunk in chunks:
                data.append((chunk, score))
        return data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        chunk, score = self.data[idx]
        inputs = self.tokenizer(chunk, padding='max_length', truncation=True, max_length=self.max_length, return_tensors="pt")
        return {
            'input_ids': inputs['input_ids'].squeeze(0),
            'attention_mask': inputs['attention_mask'].squeeze(0),
            'score': torch.tensor(score, dtype=torch.float)
        }

    def split_into_chunks(self, text):
        words = text.split()
        chunks = []
        start = 0
        while start < len(words):
            end = min(start + self.chunk_size, len(words))
            chunk = ' '.join(words[start:end])
            chunks.append(chunk)
            if end == len(words):
                break
            start += self.chunk_size - self.overlap
        return chunks

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def load_data(file_path, with_labels=True):
    df = pd.read_csv(file_path)
    texts = df['transcription'].tolist()
    if with_labels:
        scores = df['label'].tolist()
        return texts, scores
    else:
        filenames = df['filename'].tolist()
        return filenames, texts

# Training function
def train_model(model, train_loader, val_loader, epochs, learning_rate, device):
    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
    loss_fn = torch.nn.MSELoss()

    for epoch in range(epochs):
        model.train()
        total_loss = 0
        train_preds = []
        train_scores = []

        for batch in train_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            scores = batch['score'].to(device)
            if scores.dim() == 0:
                scores = scores.unsqueeze(0)

            optimizer.zero_grad()
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits.squeeze()
            if logits.dim() == 0:
                logits = logits.unsqueeze(0)
            loss = loss_fn(logits, scores)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

            train_preds.extend(logits.detach().cpu().numpy())
            train_scores.extend(scores.cpu().numpy())

        avg_train_loss = total_loss / len(train_loader)
        train_mse = mean_squared_error(train_scores, train_preds)
        train_acc = accuracy_score(
            [round(p) for p in train_preds],
            [round(s) for s in train_scores]
        )

        print(f"Epoch {epoch+1}/{epochs} | Train Loss: {avg_train_loss:.4f} | Train MSE: {train_mse:.4f} | Train Accuracy: {train_acc:.4f}")

        # Validation
        model.eval()
        val_scores = []
        val_preds = []
        with torch.no_grad():
            for batch in val_loader:
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                scores = batch['score'].to(device)

                outputs = model(input_ids=input_ids, attention_mask=attention_mask)
                logits = outputs.logits.squeeze()
                val_preds.extend(logits.cpu().numpy())
                val_scores.extend(scores.cpu().numpy())
                val_acc = accuracy_score(
                    [round(p) for p in val_preds],
                    [round(s) for s in val_scores]
                )

        val_mse = mean_squared_error(val_scores, val_preds)
        print(f"Validation MSE: {val_mse:.4f} | Validation Accuracy: {val_acc:.4f}")

In [ ]:

# Load training data
train_texts, train_scores = load_data('train_ready.csv')

# Split into training and validation sets
train_texts, val_texts, train_scores, val_scores = train_test_split(train_texts, train_scores, test_size=0.1, random_state=42)

# Create datasets and dataloaders
train_dataset = GrammarDataset(train_texts, train_scores, tokenizer, MAX_LENGTH, CHUNK_SIZE, OVERLAP)
val_dataset = GrammarDataset(val_texts, val_scores, tokenizer, MAX_LENGTH, CHUNK_SIZE, OVERLAP)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

# Train the model
train_model(model, train_loader, val_loader, EPOCHS, LEARNING_RATE, DEVICE)

    # Save the trained model
model.save_pretrained('grammar_score_model')
tokenizer.save_pretrained('grammar_score_model')

Epoch 1/5 | Train Loss: 4.0392 | Train MSE: 4.0819 | Train Accuracy: 0.1940
Validation MSE: 1.7786 | Validation Accuracy: 0.2500
Epoch 2/5 | Train Loss: 1.8365 | Train MSE: 1.8583 | Train Accuracy: 0.3048
Validation MSE: 1.4395 | Validation Accuracy: 0.4423
Epoch 3/5 | Train Loss: 1.4699 | Train MSE: 1.4880 | Train Accuracy: 0.3464
Validation MSE: 1.4481 | Validation Accuracy: 0.5000
Epoch 4/5 | Train Loss: 1.1961 | Train MSE: 1.1985 | Train Accuracy: 0.4018
Validation MSE: 1.2458 | Validation Accuracy: 0.4423
Epoch 5/5 | Train Loss: 0.8889 | Train MSE: 0.8986 | Train Accuracy: 0.4596
Validation MSE: 1.2350 | Validation Accuracy: 0.4615


('grammar_score_model/tokenizer_config.json',
 'grammar_score_model/special_tokens_map.json',
 'grammar_score_model/vocab.json',
 'grammar_score_model/merges.txt',
 'grammar_score_model/added_tokens.json')

In [ ]:
# Load test data without labels
test_df = pd.read_csv('test_ready.csv')
test_filenames = test_df['filename'].tolist()
test_texts = test_df['transcription'].tolist()

# Dummy scores (not used)
dummy_scores = [0.0] * len(test_texts)

# Create test dataset and loader
test_dataset = GrammarDataset(test_texts, dummy_scores, tokenizer, MAX_LENGTH, CHUNK_SIZE, OVERLAP)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

# Predict and collect per-chunk scores
model.eval()
all_preds = []
with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].to(DEVICE)
        attention_mask = batch['attention_mask'].to(DEVICE)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits.squeeze()

        if logits.dim() == 0:
            all_preds.append(logits.item())
        else:
            all_preds.extend(logits.cpu().numpy())

# Group predictions by original test texts (accounting for chunking)
chunk_counts = []
for text in test_texts:
    if not isinstance(text, str) or not text.strip():
        chunk_counts.append(1)  # handle empty or bad text with 1 dummy chunk
    else:
        temp_dataset = GrammarDataset([text], [0.0], tokenizer, MAX_LENGTH, CHUNK_SIZE, OVERLAP)
        chunk_counts.append(len(temp_dataset))


final_preds = []
i = 0
for count in chunk_counts:
    preds = all_preds[i:i + count]
    avg_pred = sum(preds) / len(preds)
    final_preds.append(min(5, max(1, round(avg_pred))))  # Clamp between 1 and 5
    i += count

# Create submission DataFrame
submission_df = pd.DataFrame({'filename': test_filenames, 'label': final_preds})
submission_df.to_csv('submission.csv', index=False)
print("Submission file saved as 'submission.csv'")


✅ Submission file saved as 'submission.csv'


In [ ]:
submission_df

,filename,label
0,audio_706.wav,3
1,audio_800.wav,3
2,audio_68.wav,4
3,audio_1267.wav,3
4,audio_683.wav,3
...,...,...
190,audio_135.wav,4
191,audio_512.wav,5
192,audio_529.wav,5
193,audio_762.wav,5
